<a href="https://colab.research.google.com/github/koshal123/deep-learning-challenge/blob/master/grand_challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importing Test/Train Data from Google Drive

In [1]:
 !pip install -U -q PyDrive ## you will have install for every colab session

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)   

     |████████████████████████████████| 993kB 9.3MB/s 


In [0]:
#Accessing the training data
training_data = drive.CreateFile({'id':'1wfT8FqrxTpf-384jH1S9ec53uk1cktp9'})
training_data.GetContentFile('training_set.zip')
!unzip training_set.zip 

In [0]:
#Accessing the test data
test_data = drive.CreateFile({'id':'1S7QNdVXH9LHUvT29iVpSv9DQQGu4qefE'})
test_data.GetContentFile('test_set.zip')
!unzip test_set.zip 

#Importing pixel size information of training set and test set from system

In [4]:
#importing test set pixel size .csv file from system
from google.colab import files
uploaded = files.upload() 

Saving test_set_pixel_size.csv to test_set_pixel_size.csv


In [0]:
import pandas as pd
test_pixel= pd.read_csv('test_set_pixel_size.csv')  

In [6]:
#importing training set pixel size .csv file from system
from google.colab import files 
uploaded = files.upload()

Saving training_set_pixel_size_and_hc.csv to training_set_pixel_size_and_hc.csv


In [0]:
training_pixel = pd.read_csv('training_set_pixel_size_and_hc.csv') 


#Setting cuda as our device, Dynamically by using code:

In [0]:
import torch

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


#Creating mask generation for whole training set

mask generation is only once,,  as otherwise images were unnecessary repeats

In [10]:
#mask generation is only once
#as otherwise images were unnecessary repeats

''''import os 
import cv2

def mask_generator(source_file_path, dest_file_path):
    file_names = sorted([file for _,_,files in os.walk(source_file_path) for file in files])
    for i, file_name in enumerate(file_names):
        if i % 2 != 0:
            im = cv2.imread(source_file_path+file_name)
            imgray = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
            ret,thresh = cv2.threshold(imgray,127,255,0)
            im2, contours, hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
            ellipse = cv2.fitEllipse(contours[0])
            im1 = cv2.ellipse(im,ellipse,(255,255,255),-1)
            image_name = file_name.replace('Annotation', 'Mask')
            cv2.imwrite(dest_file_path+image_name, im1)
    
    return 1

SyntaxError: ignored

In [0]:
#mask_generator('/content/training_set/','/content/drive/My Drive/Mask_HC18/')

In [0]:
#!pip install mlflow
#import mlflow
#import mlflow.pytorch

In [0]:
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook 
import copy
import glob

import torch
import torchvision
from torch.utils.data import DataLoader, Dataset, random_split
import torchvision.transforms as transforms
from torchvision import datasets

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import numpy as np
import pandas as pd

#changing dataset to Torch form

In [0]:
import numpy as np
import torch.utils.data
from PIL import Image
import glob

class CHALLENGE18(torch.utils.data.Dataset):
  def __init__(self, png_images, tranform=None,train=True):
    self.X_train = glob.glob('/content/training_set/*HC.png')
    
    self.Y_train = glob.glob('/content/drive/My Drive/Mask_HC18/*')
    
    
   
    
    self.png_images = png_images
    
  def __len__(self):
    if self.png_images == "train":
      return len(self.X_train)
    else:
      print("code is wrong")
      
  def __getitem__(self,idx):
    if self.png_images == "train":
      
      X=np.array(Image.open(self.X_train[idx]).resize((572,572))).reshape(1,572,572) #as per U-net paper
      
      Y=np.array(Image.open(self.Y_train[idx]).convert('L').resize((572,572))).reshape(1,572,572)
      
      
      print(X.shape, Y.shape)
      
      return torch.from_numpy(X).float(), torch.from_numpy(Y).float()
    
    else:
      print("code is wrong")
    



In [0]:
total_images = CHALLENGE18("train", train=True)


In [0]:
#total_images[5]

In [0]:
train_images = int(0.7 * len(total_images))

test_images = len(total_images)- train_images

train_data, validation_data = random_split(total_images,[train_images, test_images])

In [0]:
#train_data[2]

In [0]:
batch_size = 2

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
validation_loader = torch.utils.data.DataLoader(validation_data, batch_size=batch_size, shuffle=True)

test_data = CHALLENGE18("test",train=False)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size,shuffle=False)

In [0]:
def imshow(img, title):
    img = img.clone().detach().numpy()
    #npimg = img * ((0.5,0.5,0.5)) + 0.5   # un-normalizing the image
    #plt.figure(figsize=(batch_size*5, 5))
    #plt.axis('off')
    #img = np.transpose(img, (1, 2, 0))
    print('npimage shape',img.shape)
    #plt.imshow(npimg)
    #plt.title(title)
    plt.show()
    return img

In [0]:
def show_train_batch_images(dataloader):
    x, y = next(iter(dataloader))
    print(x.shape)
    return imshow(x[0],'d')
    
    #x = torchvision.utils.make_grid(x)
    #y = torchvision.utils.make_grid(y)
    #imshow(x, title=[n for n in xf])
    #imshow(y, title=[n for n in yf])

In [88]:
p =show_train_batch_images(validation_loader)

(1, 572, 572) (1, 572, 572)
(1, 572, 572) (1, 572, 572)
torch.Size([2, 1, 572, 572])
npimage shape (1, 572, 572)


In [0]:
#q= show_train_batch_images(train_loader)

In [90]:
p.shape

(1, 572, 572)

In [91]:
type(p.reshape(572,572))

numpy.ndarray

In [0]:
#plt.imshow(p)

#prepare for U-net

In [0]:
#as in the dataset, rarely any image has features in the boundary, so I am not considering the padding

class UNet(nn.Module):
  def __init__(self):
    super(UNet, self).__init__()
    self.direct_1 = self.direct(1)
    self.down_1 = self.down(64)   #as per UNet paper, no of channels doubles per pooling in downside
    self.down_2 = self.down(128)
    self.down_3 = self.down(256)
    self.lowest_4 = self.lowest(512)


    self.up_layer_4 = self.up_conv(1024)
    self.up_4 = self.up(1024)
    
    self.up_layer_3 = self.up_conv(512)
    self.up_3 = self.up(512)
    
    self.up_layer_2 = self.up_conv(256)
    self.up_2 = self.up(256)
    
    self.up_layer_1 = self.up_conv(128)
    self.up_1 = self.up(128)
    
    self.final_output = nn.Conv2d(64,2, 1, 1)   #kernel size = 1 and stride = 1
    
    
    
    
    
#function for first convolutional layer   

  def direct(self,input_channels):
    return nn.Sequential(
        nn.Conv2d(1,64, 3, 1,0),    #(input_channel= 1, output_channel = 64, kernel_size = 3, stride = 1)
        nn.ReLU(inplace = True),
        nn.Conv2d(64,64, 3, 1,0),
        nn.ReLU(inplace = True)
    )   

# function for pooling 

  def pooling(self):
    return nn.MaxPool2d(2,2)   #max-pool-kernel(2*2)

#function for the contracting downside layers

  def down(self, input_channels):
    return nn.Sequential(
        nn.Conv2d(input_channels, int(2* input_channels), 3, 1),
        nn.ReLU(inplace = True),
        nn.Conv2d(int(2*input_channels), int(2* input_channels), 3, 1),
        nn.ReLU(inplace = True)
    )

#function for lowest layer

  def lowest(self, input_channels):
    return nn.Sequential(
        nn.Conv2d(input_channels, 2* input_channels, 3, 1),
        nn.ReLU(inplace = True),
        nn.Conv2d(2* input_channels, 2* input_channels, 3, 1),
        nn.ReLU(inplace = True)
    )

    
#function for upward layer

  def up(self, input_channels):
    return nn.Sequential(
        nn.Conv2d(int(input_channels), int(0.5* input_channels), 3, 1),
        nn.ReLU(inplace = True),
        nn.Conv2d(int(0.5* input_channels), int(0.5* input_channels), 3, 1), #no change in channels
        nn.ReLU(inplace = True)
    )

  #def up_conv(self, input_channels):
    #return nn.ConvTranspose2d(input_channels, int(0.5 * input_channels), 2,  2)  #feture detector has size= (2,2)
      
    
  def up_conv(self, input_channels):   #kernel size=2, stride =1
    return nn.ConvTranspose2d(input_channels, input_channels, 2,  2)  #feture detector has size=2, and stride= 1
                      
#function for copy and crop
  def crop_concat(self, x1, x2):
    
    in_width = x1.shape[-1]
    out_width = x2.shape[-1]
    crop_width = (in_width - out_width) // 2
    in_height = x1.shape[-2]
    out_height = x2.shape[-2]
    crop_height = (in_height - out_height) // 2
  
    x1 = x1[:, :, crop_height:(in_height-crop_height), crop_width:(in_width-crop_width)]
    
    return torch.cat([x1, x2],1)
    
     
  def forward(self, x):
    
#downside movement of layers
    x1 = self.direct_1(x)
    print(x1.shape)
    x2 = self.down_1(self.pooling()(x1))
    x3 = self.down_2(self.pooling()(x2))
    x4 = self.down_3(self.pooling()(x3))
    x5 = self.lowest_4(self.pooling()(x4))
    print(x5.shape) 
#upside movement
    x4_u = self.up_layer_4(x5)
    
    x4_u = self.up_4(self.crop_concat(x4, x4_u))
    print(x4_u.shape)
    x3_u = self.up_layer_3(x4_u)
    x3_u = self.up_3(self.crop_concat(x3, x3_u))
    
    x2_u = self.up_layer_2(x3_u)
    x2_u = self.up_2(self.crop_concat(x2, x2_u))
    
    #print(x2_u.shape)
    
    x1_u = self.up_layer_1(x2_u)
    x1_u = self.up_1(self.crop_concat(x1, x1_u))
    
    y= self.final_output(x1_u)
    return y
    
  
unet = UNet()
unet= unet.to(device)
  

In [0]:
import torch.optim as optim

criterion = nn.BCELoss()
optimizer = optim.SGD(unet.parameters(), lr=0.01, momentum= 0.9)

#Training

In [0]:

for epoch in range(5):
  running_loss = 0.0
  for i, data in enumerate(train_loader, 0):
    
    inputs = inputs.to(device)
    
    
    inputs, labels = data
    
    optimizer.zero_grad()
    
    outputs = unet(inputs)
    
    outputs = outputs.to(device)
    
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    
    
    
    running_loss += loss.item()
    if i % 2000 == 1999:
      print('[%d, %5d] loss:%.3f' % (epoch + 1, i +1, running_loss / 2000))
      
      running_loss = 0.0
      
print("training is finished")

#Dice coefficient

In [0]:
def dice_coeff(pred, target):
    smooth = 1.
    num = pred.size(0)
    m1 = pred.view(num, -1)  # Flatten
    m2 = target.view(num, -1)  # Flatten
    intersection = (m1 * m2).sum()

    return (2. * intersection + smooth) / (m1.sum() + m2.sum() + smooth)
       

class SoftDiceLoss(nn.Module):
    def __init__(self, inputs, targets):
        super(SoftDiceLoss, self).__init__()
        
        inputs = inputs.to(device)
        targets=targets.to(device)
        
        
        outputs = model(inputs)
        outputs = outputs.argmax(1).float()
        
        height, width = outputs.shape[1:]
        
        for output, target in zip(outputs, targets):
          diceCoeff += dice_coeff(output.view(1, 1, height, width), target.view(1, 1, height, width))
          
          
          diceCoeff /= len(dataloader.dataset)
          
          return diceCoeff

    #def forward(self, logits, targets):
    #    probs = F.sigmoid(logits)
     #   num = targets.size(0)  # Number of batches

#        score = dice_coeff(probs, targets)
 #       score = 1 - score.sum() / num
  #      return score